In [71]:
import numpy as np

### Open Serial Port
You might need to change 'COM'

In [77]:
import serial as ser
s = ser.Serial ('COM6', baudrate = 9600, parity = 'N', bytesize = 8, stopbits = 1, timeout = None)

### Define Rotation / Transformation Functions

In [73]:
def Rx (phi):
    t = phi * np.pi / 180
    c = np.cos (t)
    s = np.sin (t)
    return np.array ([
        [1, 0,  0, 0],
        [0, c, -s, 0],
        [0, s,  c, 0],
        [0, 0,  0, 1]
    ])

def Ry (phi):
    t = phi * np.pi / 180
    c = np.cos (t)
    s = np.sin (t)
    return np.array ([
        [c, 0,  s, 0],
        [0, 1,  0, 0],
        [-s, 0, c, 0],
        [0, 0,  0, 1]
    ])

def Rz (phi):
    t = phi * np.pi / 180
    c = np.cos (t)
    s = np.sin (t)
    return np.array ([
        [c, -s,  0, 0],
        [s, c,  0, 0],
        [0, 0, 1, 0],
        [0, 0,  0, 1]
    ])

def Tx (x):
    return np.array ([
        [1, 0, 0, x],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def Ty (y):
    return np.array ([
        [1, 0, 0, 0],
        [0, 1, 0, y],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def Tz (z):
    return np.array ([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])

In [64]:
def move(a):
    
    #custom start point
    #startpoint = np.array([90,85,175,180,90,65])
    startpoint= np.array([90, 90, 93, 90, 90, 65])
    a_calc = startpoint + a
    #print(a)
    command = f'{-a_calc[0]} b {a_calc[1]} s {a_calc[2]} e {a_calc[3]} v {a_calc[4]} w {a[5]} g\r\n'
    res = Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(a[3]) @ Tz(60) @ Tz(132)
    
    if res[2,3] < 0:
       raise Exception(f'Illegal z value: {res[2,3]} lower than 0')
    else:
        s.write(command.encode())
        return res
    


In [29]:
def move_nocheck(a):
    
    #custom start point
    #startpoint = np.array([90,85,175,180,90,65])
    startpoint= np.array([90, 90, 87, 90, 90, 65])
    a_calc = startpoint + a
    #print(a)
    command = f'{a_calc[0]} b {a_calc[1]} s {a_calc[2]} e {a_calc[3]} v {a_calc[4]} w {a[5]} g\r\n'
    res = Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(a[3]) @ Tz(60) @ Tz(132)

    s.write(command.encode())
    return res
    


In [15]:
Rz(-90)@Tz(40)

#1. Rot z Base 
#2. Tz 71.5mm
#3  Rot y Shoulder
#4  Tz 125mm
#5  Rot y Elbow
#6  Tz 125mm
#7  Rot y Wrist
#8  Tz 60mm
#9 (Wrist Rot)
#10 Tz 132mm
#11 Tx OFFSET 12.5mm for new start position

#def transformation(a):
#    return Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(a[3]) @ Tz(60) @ Tz(132)

In [136]:
def Move (a):
    startpoint= np.array([[90], [90], [87], [90], [90], [65]])
    print (a.shape)
    a_calc = np.copy (startpoint)
    a_calc [0:3,0:1] += a
    print (a_calc)
    command = f'{-a_calc[0,0]} b {a_calc[1,0]} s {a_calc[2,0]} e {a_calc[3,0]} v {a_calc[4,0]} w {a_calc[5,0]} g\r\n'
    res = VK (a)
    print (res)
    if res[2,3] < 0:
       raise Exception(f'Illegal z value: {res[2,3]} lower than 0')
    else:
        s.write(command.encode())

In [137]:
def VK (a):
    res = Tx(-125) @ Rz(-a[0,0]) @ Tz(71.5) @ Ry(a[1,0]) @ Tz(125) @ Ry(a[2,0]) @ Tz(125) @ Ry(0) @ Tz(60) @ Tz(132)
    return res;

In [155]:
def Diff (a):
    pos = VK (a)
    target = np.array ([[180.0],[-50.0],[380.0]])
    return pos[0:3,3:4] - target

In [157]:
B=10
S=45
E=0

a = np.array([[B],[S],[E]])
print (VK (a))
print (Diff (a))

# print(f(np.array([B,S,E])))
#Move(a)
#transformation(a)

[[ 6.96364240e-01  1.73648178e-01  6.96364240e-01  1.82792994e+02]
 [-1.22787804e-01  9.84807753e-01 -1.22787804e-01 -5.42722094e+01]
 [-7.07106781e-01  0.00000000e+00  7.07106781e-01  3.84041197e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 2.79299422]
 [-4.27220935]
 [ 4.04119728]]


In [192]:
def j(f,x0):
    h=1E-8
    y0 = f(x0)
    
    col1 = np.copy(x0.astype(float))
    col1[0,0] += h
    col1_y = f(col1)
    col1_df = (col1_y - y0) / h

    col2 = np.copy(x0.astype(float))
    col2[1] += h
    col2_y = f(col2)
    col2_df = (col2_y - y0) / h
    
    col3 = np.copy(x0.astype(float))
    col3[2] += h
    col3_y = f(col3)
    col3_df = (col3_y - y0) / h
    
    return np.concatenate((col1_df,col2_df,col3_df),axis=1)

#Iterative newton-raphson approximation for angle correction
#Input: phi1,phi2,phi3 (Winkel!)
def approximate(f,x0):
    x = np.copy(x0.astype(float))
    print('bli', j(f,x))
    for i in range(5):
        #x -= np.linalg.solve(j(x.reshape(3,)),f(x.reshape(3,)))
        x -= np.linalg.inv(j(f,x)) @ f(x)
        print(f'n={i+1}\n{x}\n')
    return x
        
anew = approximate(Diff,a)
print (VK (anew))

bli [[-0.94723021  5.37199867  3.85276735]
 [-5.37200151 -0.94722878 -0.67934707]
 [ 0.         -5.45487637 -3.91220851]]
n=1
[[ 9.40279558e+00]
 [-3.07565712e+06]
 [ 4.28851858e+06]]

n=2
[[ 9.14092127e+00]
 [-3.07597347e+06]
 [ 4.28870871e+06]]

n=3
[[ 9.00942800e+00]
 [-3.07677351e+06]
 [ 4.28979109e+06]]

n=4
[[ 1.01366013e+01]
 [-3.07743930e+06]
 [ 4.29059812e+06]]

n=5
[[ 1.11467961e+01]
 [-3.07756927e+06]
 [ 4.29079375e+06]]

[[ 8.92971853e-01  1.93323370e-01  4.06481666e-01  1.19623047e+02]
 [-1.75951639e-01  9.81135095e-01 -8.00933589e-02 -4.82006524e+01]
 [-4.14297346e-01  0.00000000e+00  9.10141587e-01  4.01274340e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [166]:
### Preset Demo

move(np.array([0,0,0,0,0,0]))
#move_nocheck(np.array([0,-90,-90,-30,0,0]))

array([[   1. ,    0. ,    0. , -125. ],
       [   0. ,    1. ,    0. ,    0. ],
       [   0. ,    0. ,    1. ,  513.5],
       [   0. ,    0. ,    0. ,    1. ]])

In [5]:
def rad(phi):
    return phi * np.pi / 180
def c(phi):
    return np.cos(phi)
def s(phi):
    return np.sin(phi)

def dh_calc(a,alpha,d,theta):
    return np.array([
        [c(theta), -s(theta), 0, a],
        [s(theta)*c(alpha), c(theta)*c(alpha), -s(alpha), -d * s(alpha)],
        [s(theta)*s(alpha), c(theta)*s(alpha), c(alpha), d * c(alpha)],
        [0,0,0,1]
    ])

In [10]:
b_a = rad(-90)
r1 = np.array([
    [c(b_a),-s(b_a),0,0],
    [s(b_a),c(b_a),0,0],
    [0,0,1,0],
    [0,0,0,1]
])

t1 = np.array([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,40],
    [0,0,0,1]
])



array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

### Warning: Experimental Code for DH
Never tested and probably not working

In [15]:
#warning: rad
#Ansatz DH, nicht überprüft

a1 = rad(90)
a2 = rad(90)
a3 = rad(180)
a4 = rad(0)
a5 = rad(90)


dh_parameters = np.array ([
    [0, rad(90), 71.5, a1],
    [125, 0, 0, a2+rad(90)],
    [125, 0, 0, a3],
    [0, rad(90), 0, a4 - rad(90)],
    [0, 0, 192, a5]
])

dh_matrices = []

for row in dh_parameters:
    dh_matrices.append(dh_calc(row[0],row[1],row[2],row[3]))
    


dh_matrices[0]@dh_matrices[1]@dh_matrices[2]@dh_matrices[3]@dh_matrices[4]

array([[ 3.06161700e-16, -6.12323400e-17,  1.00000000e+00,
         1.92000000e+02],
       [ 6.12323400e-17, -1.00000000e+00, -6.12323400e-17,
        -7.15000000e+01],
       [ 1.00000000e+00,  6.12323400e-17, -3.06161700e-16,
        -5.87830464e-14],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

## Reverse Kinematics

In [23]:
### target = np.array([x,y,z])
target = np.array([240,50,10])
#target = np.array([125,0,0])

h = 1E-8

def f(a):
    
    #Implement three axis of freedom
    res = Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(0) @ Tz(60) @ Tz(132)
    
    #res = Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(a[3]) @ Tz(60) @ Tz(132)
    
    delta = target.reshape(3,1) - res[0:3,3:4]
    #print(f'i calc {delta} = {target.reshape(3,1)} - {res[0:3,3:4]}')
    return delta

def f_d(x0):
    y0 = f (x0)
    #print (y0)
    x1 = np.copy (x0); x1 [0] += h; y1 = f (x1); df1 = (y1 - y0) / h;
    x2 = np.copy (x0); x2 [1] += h; y2 = f (x2); df2 = (y2 - y0) / h;
    x3 = np.copy (x0); x3 [2] += h; y3 = f (x3); df3 = (y3 - y0) / h;
    J = np.concatenate ((df1, df2, df3), axis = 1)
    dx = np.linalg.solve (J, y0)
    print (dx)

def rev_k(x0):
    y0 = f(x0)
    
    col1 = np.copy(x0)
    col1[0] += h
    col1_y = f(col1)
    col1_df = (col1_y - y0) / h

    col2 = np.copy(x0)
    col2[1] += h
    col2_y = f(col2)
    col2_df = (col2_y - y0) / h
    
    col3 = np.copy(x0)
    col3[2] += h
    col3_y = f(col3)
    col3_df = (col3_y - y0) / h
    
    J = np.concatenate((col1_df,col2_df,col3_df),axis=1)
    
    dx = np.linalg.solve (J,y0)
    print(dx)
    print(x0)


#Jacobimatrix
def j(x0):
    y0 = f(x0)
    
    col1 = np.copy(x0)
    col1[0] += h
    col1_y = f(col1)
    col1_df = (col1_y - y0) / h

    col2 = np.copy(x0)
    col2[1] += h
    col2_y = f(col2)
    col2_df = (col2_y - y0) / h
    
    col3 = np.copy(x0)
    col3[2] += h
    col3_y = f(col3)
    col3_df = (col3_y - y0) / h
    
    return np.concatenate((col1_df,col2_df,col3_df),axis=1)

#Iterative newton-raphson approximation for angle correction
#Input: phi1,phi2,phi3 (Winkel!)
def approximate(x0):
    x = np.copy(x0).reshape(3,1)
    for i in range(15):
        #x -= np.linalg.solve(j(x.reshape(3,)),f(x.reshape(3,)))
        x -= np.linalg.inv(j(x.reshape(3,))) @ f(x.reshape(3,))
        print(f'n={i+1}\n{x}\n')

#approximate(np.array([10,10,20]).astype(float))
f(np.array([10,10,20]).astype(float))
#rev_k(np.array([20,10,20]).astype(float))

array([[ 187.53171219],
       [  81.29244736],
       [-459.13102213]])

In [193]:
s.close()